---
title: "Matematyczne modelowanie procesów decyzyjnych w finansach"
author: "Rafał Głodek, Joanna Kusy, Oliwia Makuch, Tomasz Srebniak"
execute: 
  echo: true
format:
  revealjs:
    theme: serif
    transition: slide
    scrollable: true
    self-contained: true
title-slide-attributes:
    data-background-image: "img_prezentacja2.jpg"
    data-background-size: cover
    data-background-opacity: "0.65"
    data-background-color: "#000000"
---


## Powtórzenie

Model GARCH wykorzystywany jest przy modelowaniu i przewidywaniu zmienności w szeregach czasowych. Stosuje się go głównie w analizie finansowej.\

Wzór ogólny

$$
GARCH(p, q): \sigma_t^2=\omega+\sum_{i=1}^{p}{\alpha_i\epsilon_{t-i}^2} + \sum_{j=1}^{q}{\beta_j\sigma_{t-j}^2}
$$

Najczęściej omawiany przypadek

$$
GARCH(1, 1): \sigma_t^2=\omega+{\alpha\epsilon_{t-1}^2} + {\beta\sigma_{t-1}^2}
$$

$\omega$ - bazowy poziom wariancji warunkowej

$\alpha$ – współczynnik reakcji na nowe informacje (efekt ARCH)

$\beta$ – współczynnik pamięci zmienności (efekt GARCH)

$\epsilon_{t-1}^2$ – kwadrat błędu z kroku $t-1$

$\sigma_{t-1}^2$ – wariancja z kroku $t-1$


In [ ]:
#| echo: false
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from arch import arch_model
from arch.univariate import SkewStudent

In [ ]:
df = pd.read_csv('/Users/joannakusy/pwr/rok3/analiza_danych_ankietowych/Categorical_Data_Analysis/matematyka_dla_przemyslu/Prezentacja4/Dane - Kaggle/aapl.us.txt')
df.set_index('Date', inplace=True)

# Liczenie zwrotów procentowych
returns = df['Close'].pct_change().dropna()

normal_gm = arch_model(returns, p = 1, q = 1, 
mean = 'constant', 
vol = 'GARCH', 
dist = 'normal')

gm_result = normal_gm.fit()

## Założenia dotyczące rozkładu standaryzowanych reszt

$$
\text{reszta} = \epsilon_t = \text{predyktowany zwrot} - \text{średni zwrot}
$$

$$
\text{standaryzowana reszta} = \frac{\epsilon_t}{\sigma_t}
$$


In [ ]:
gm_resid = gm_result.resid
gm_std = gm_result.conditional_volatility

gm_std_resid = gm_resid / gm_std

\

-   Rozkład normalny (domyślna opcja): `"normal"`

-   Rozkład t-Studenta -- grube ogony rozkładu: `"t"`

-   Skośny rozkład t-Studenta -- grube ogony i skośność: `"skewt"`

Implementacja


In [ ]:
normal_gm = arch_model(
  returns, p = 1, q = 1, 
  mean = 'constant', 
  vol = 'GARCH', 
  dist = 'normal'  # założenie o rozkładzie
  )

In [ ]:
normal_resid = np.random.normal(0, 1, len(gm_std_resid))

# tstudent
t_gm = arch_model(returns, p = 1, q = 1, 
mean = 'constant', 
vol = 'GARCH', 
dist = 't')

gm_t_result = t_gm.fit()

gm_t_resid = gm_t_result.resid
gm_t_std = gm_t_result.conditional_volatility

gm_t_std_resid = gm_t_resid /gm_t_std

df_hat = gm_t_result.params['nu']
t_resid = np.random.standard_t(df=df_hat, size=len(gm_t_std_resid))
t_resid_scaled = t_resid / np.sqrt(df_hat / (df_hat - 2))

# skośny tstudent
skewt_gm = arch_model(returns, p = 1, q = 1, 
mean = 'constant', 
vol = 'GARCH', 
dist = 'skewt')

gm_skewt_result = skewt_gm.fit()

gm_skewt_resid = gm_skewt_result.resid
gm_skewt_std = gm_skewt_result.conditional_volatility

gm_skewt_std_resid = gm_skewt_resid /gm_skewt_std

df_hat_skewt = gm_skewt_result.params['eta']
skew_hat = gm_skewt_result.params['lambda']

skewt = SkewStudent()
f_skew_resid = skewt.simulate((df_hat_skewt, skew_hat))
skewt_resid = f_skew_resid(len(gm_skewt_std_resid))

------------------------------------------------------------------------

### Rozkład normalny standaryzowanych reszt


In [ ]:
#| fig-align: "center"
plt.hist(gm_std_resid, bins = 50, 
         facecolor = 'orange', label = 'Standardized residuals')
plt.hist(normal_resid, bins = 50, 
         facecolor = 'tomato', label = 'Normal residuals', alpha=0.7)
plt.legend(loc = 'upper left')
plt.show()

### Rozkład t--Studenta standaryzowanych reszt


In [ ]:
#| fig-align: "center"
plt.hist(gm_t_std_resid, bins = 50, 
         facecolor = 'orange', label = 'Standardized residuals')
plt.hist(t_resid_scaled, bins = 50, 
         facecolor = 'tomato', label = 't-Student residuals', alpha=0.7)
plt.legend(loc = 'upper left')
plt.show()

### Skośny rozkład t--Studenta standaryzowanych reszt


In [ ]:
#| fig-align: "center"
plt.hist(gm_skewt_std_resid, bins = 50, 
         facecolor = 'orange', label = 'Standardized residuals')
plt.hist(skewt_resid, bins = 50, 
         facecolor = 'tomato', label = 'skewt residuals', alpha=0.7)
plt.legend(loc = 'upper left')
plt.show()

## Założenia o średniej

-   Stała średnia (domyślna opcja): `"constant"`

-   Zerowa średnia: `"zero"`

-   Średnia modelowana jako proces AR: `"AR"` (np. AR(1), AR(2), ...)

Implementacja


In [ ]:
ar_gm = arch_model(
  returns, p = 1, q = 1, 
  mean = 'AR', lags = 1,  # założenie o średniej
  vol = 'GARCH', 
  dist = 'normal' 
  )

In [ ]:
constant_gm = arch_model(
  returns, p = 1, q = 1,
  mean = 'constant',
  vol = 'GARCH',
  dist = 'normal'
  )
zero_gm = arch_model(
  returns, p = 1, q = 1,
  mean = 'zero',
  vol = 'GARCH',
  dist = 'normal'
  )
cmean_result = constant_gm.fit()
armean_result = ar_gm.fit()
zero_result = zero_gm.fit()

cmean_vol = cmean_result.conditional_volatility
armean_vol = armean_result.conditional_volatility
zeromean_vol = zero_result.conditional_volatility

## Wpływ założeń o średniej

... na szacowaną zmienność warunkową


In [ ]:
#| fig-align: "center"
plt.plot(cmean_vol, color = 'blue', label = 'Constant Mean Volatility')
plt.plot(armean_vol, color = 'red', label = 'AR Mean Volatility', alpha=0.7)
plt.plot(zeromean_vol, color = 'green', label = 'Zero Mean Volatility', alpha=0.7, linestyle='dashed')
plt.legend(loc = 'upper right')
plt.xlabel('')
plt.savefig('mean_volatility.png')

Korelacja między wynikami


In [ ]:
# Łączymy w macierz (3 x n)
all_vols = np.vstack([cmean_vol, armean_vol, zeromean_vol])

# Usuwamy kolumny zawierające NaN
valid = ~np.isnan(all_vols).any(axis=0)

# Liczymy korelację tylko na poprawnych danych
corr = np.corrcoef(all_vols[:, valid])
corr_df = pd.DataFrame(corr, 
                       columns=['Constant Mean', 'AR Mean', 'Zero Mean'], 
                       index=['Constant Mean', 'AR Mean', 'Zero Mean'])
corr_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'center'), ('font-size', '23px')]},
     {'selector': 'td', 'props': [('text-align', 'center'), ('font-size', '23px')]}]
).set_properties(**{
    'margin-left': 'auto',
    'margin-right': 'auto',
    'font-size': '10px',
    'background-color': 'transparent',
    'color': 'black'  # lub 'white', jeśli tło slajdu jest ciemne
})

## Modele dla asymetrycznych wahań zmienności

Model GARCH zakłada, że zmienność reaguje symetrycznie na zdarzenia rynkowe. W praktyce jednak negatywne informacje (np. spadki cen) wywołują silniejszy wzrost zmienności niż pozytywne.

![](symmetry_vs_asymmetry.png){fig-align="center"}

### GJR--GARCH(p, o, q)

-   umożliwia większy wpływ negatywnych szoków na zmienność poprzez dodatkowy warunkowy składnik $(\gamma > 0)$

    $$
    \sigma^2 = \omega + \sum_{i=1}^p\alpha_i\epsilon_{t-i}^2+\sum_{j=1}^o\gamma_jI_{\{\epsilon_{t-j}<0\}}\epsilon_{t-j}^2+\sum_{k=1}^q\beta_k\sigma_{t-k}^2
    $$

Implementacja


In [ ]:
arch_model(
  returns, p = 1, q = 1, 
  o = 1,  # ustawiamy rząd składnika asymetrycznego
  mean = 'constant', 
  vol = 'GARCH'
  )

\

### EGARCH(p, o, q)

$$
\ln\sigma_{t}^{2}=\omega+\sum_{i=1}^{p}\alpha_{i}\left(\left|e_{t-i}\right|-\sqrt{2/\pi}\right)+\sum_{j=1}^{o}\gamma_{j} e_{t-j}+\sum_{k=1}^{q}\beta_{k}\ln\sigma_{t-k}^{2},
$$

gdzie $e_{t}=\epsilon_{t}/\sigma_{t}$.

-   wykładniczy GARCH

-   parametry występujące w modelu nie muszą być dodatnie

Implementacja


In [ ]:
arch_model(
  returns, p = 1, q = 1, 
  o = 1, 
  mean = 'constant', 
  vol = 'EGARCH'  # określamy model
  )

## Porównanie modeli


In [ ]:
import plotly.graph_objects as go
# Generowanie fikcyjnych danych
np.random.seed(42)

# Zmienność i dane zwrotów
returns = np.random.normal(0, 1, 100)
gm_std = np.random.normal(0.02, 0.01, 100)  # GARCH Volatility
gjrgm_vol = np.random.normal(0.015, 0.01, 100)  # GJR-GARCH Volatility
egarch_vol = np.random.normal(0.03, 0.015, 100)  # EGARCH Volatility

# Tworzymy wykres
fig = go.Figure()

# Dodajemy dane do wykresu
fig.add_trace(go.Scatter(y=returns, mode='lines', name='Returns', line=dict(color='grey'), opacity=0.4))
fig.add_trace(go.Scatter(y=gm_std, mode='lines', name='GARCH Volatility', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=gjrgm_vol, mode='lines', name='GJR-GARCH Volatility', line=dict(color='green')))
fig.add_trace(go.Scatter(y=egarch_vol, mode='lines', name='EGARCH Volatility', line=dict(color='red')))

# Aktualizujemy układ wykresu
fig.update_layout(
    title='Porównanie estymowanej zmienności',
    xaxis_title='Obserwacje',
    yaxis_title='Wartość',
    template='plotly_white',
    height=500
)